# GRU

In [3]:
import pandas as pd
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
from scipy import sparse
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostRegressor
import ngboost as ngb
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'Malgun Gothic'

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import tensorflow as tf
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

def preprocess_data():
    all_data = pd.read_csv('asos_train.csv', encoding='cp949')
    
#     ['영암에프원태양광b', '[서인천]발전부지내 태양광 1단계', '[서인천]발전부지내 태양광 1단계ESS',
#        '[서인천]발전부지내 태양광 2단계', '군산복합2단계태양광', '군산복합2단계태양광ESS', '대전 학하 연료전지',
#        '삼랑진태양광#1', '삼랑진태양광#2', '서인천연료전지', '서인천연료전지2',
#        '세종시수질복원센터태양광연계 ESS', '수질복원센터태양광', '의왕 연료전지', '장흥풍력', '천안청수연료전지']

    relevant_loc = ['영암에프원태양광b','[서인천]발전부지내 태양광 1단계','[서인천]발전부지내 태양광 2단계', 
                    '삼랑진태양광#1', '삼랑진태양광#2', '수질복원센터태양광']

    all_data = all_data[all_data['name'].isin(relevant_loc)]

    all_data.fillna(-1, inplace=True)
    
    all_data['date'] = pd.to_datetime(all_data['date'])
    all_data['year'] = all_data['date'].dt.year
    all_data['month'] = all_data['date'].dt.month
    all_data['hour'] = all_data['date'].dt.hour


    # Index(['power', 'date', 'name', 'capacities', 'asos_num', 'land', 'power/land',
    #        'location', 'Date/Time', 'Temp(C)', 'Prec(mm)', 'Wind_speed(m/s)',
    #        'Wind_Direction(16 compass points)', 'Humidity', 'Vapor_pressure(hPa)',
    #        'Dew_Point(C)', 'Local Atmospheric Pressure(hPa)',
    #        'Sea-level_Pressure(hPa)', 'sunshine(hr)', 'Solar_Radiation(MJ/m2)',
    #        'Snowfall(cm)', '3-hour_Fresh_Snowfall', 'Cloud_Cover(1/10)',
    #        'Mid-Low_Cloud_Cover', 'Cloud_Form', 'Lowest_Cloud_Height(100m)',
    #        'Visibility(10m)', 'Ground_Condition', 'Phenomenon_Number',
    #        'Ground Temp', '5cm_Underground_Temp', '10cm_Underground_Temp',
    #        '20cm_Underground_Temp', '30cm_Underground_Temp'],
    #       dtype='object')

    drop_columns = [ "asos_num", "location", "power", "capacities", "land",
                   'Wind_Direction(16 compass points)', 'Humidity', 'Vapor_pressure(hPa)', "Date/Time",
                    'Local Atmospheric Pressure(hPa)',
                   'Sea-level_Pressure(hPa)', 
                   '3-hour_Fresh_Snowfall',
                   'Mid-Low_Cloud_Cover', 'Cloud_Form', 'Lowest_Cloud_Height(100m)',
                   'Visibility(10m)', 'Ground_Condition', 'Phenomenon_Number',
                   'Ground Temp', '5cm_Underground_Temp', '10cm_Underground_Temp',
                   '20cm_Underground_Temp', '30cm_Underground_Temp']

    all_data = all_data.drop(columns=drop_columns)
    
    all_data['power/land'] = normalize_power_with_scaler(all_data[['power/land']])
    
    all_data_train = all_data[all_data['year'].isin([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])]
    all_data_test = all_data[all_data['year'].isin([2021,2022])]
    
    all_data_train.sort_values(by='date', inplace=True)
    all_data_test.sort_values(by='date', inplace=True)


    return all_data_train, all_data_test

def normalize_power_with_scaler(data):
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data)
    return normalized_data


def get_data_splits(train_scaled, test_scaled):
    train_X = train_scaled.drop(columns=['power/land', 'name', 'date'])
    test_X = test_scaled.drop(columns=['power/land', 'name', 'date'])
    train_y = train_scaled['power/land']
    test_y = test_scaled['power/land']
    #X_train.shape, y_train.shape, X_test.shape, y_test.shape
    #((39205, 16), (39205,), (4470, 16), (4470,))
    print('------done get_data_splits---------------')
    return train_X, train_y, test_X, test_y


def get_possible_combinations():
    config = [[True, False], [16, 32, 64, 128], [8, 16, 32]]
    print('------done get_possible combinations---------------')
    return list(itertools.product(*config))

def create_and_train_model(train_X, train_y, test_X, test_y, n_neurons, n_batch_size, dropout, additional_layer):
    train_X2 = train_X.values.reshape((train_X.shape[0], 1, train_X.shape[1]))
    train_y2 = train_y.values.reshape((train_y.shape[0]))

    train_X3, valid_X, train_y3, valid_y = train_test_split(train_X2, train_y2, test_size = 0.2, shuffle = False)

    test_X2 = test_X.values.reshape((test_X.shape[0], 1,test_X.shape[1]))
    test_y2 = test_y.values#.reshape((test_y.shape[0]))

    model = Sequential()
    model.add(GRU(units=n_neurons, return_sequences=True, input_shape=(1, train_X.shape[1])))
    model.add(Dropout(dropout))
    if additional_layer:
        model.add(GRU(units=n_neurons, return_sequences=True))
        model.add(Dropout(dropout))
    model.add(GRU(units=n_neurons, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units=n_neurons, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(GRU(units=n_neurons, return_sequences=False))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='linear'))

    model.compile(optimizer='adam', loss='mae', metrics=['mae', 'accuracy'])

    es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=20)
    mc = ModelCheckpoint(f_name, monitor='val_loss', mode='min', verbose=0, save_best_only=True)
    rl = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00000001)

    model.fit(train_X3, train_y3, validation_data = (valid_X, valid_y), epochs=100, verbose=1, batch_size=n_batch_size, callbacks=[es, mc, rl], shuffle=False)
    print('------done create_and_train_model---------------')
    return model

def evaluate_hyperparameters(train_X, train_y):
    possible_combinations = get_possible_combinations()
    kfold = KFold(n_splits=5, shuffle=True)
    losses = np.empty(len(possible_combinations))

    for i, (additional_layer, n_neurons, n_batch_size) in enumerate(possible_combinations):
        print('--------------------------------------------------------------------')
        print(f'Combination #{i+1}: {additional_layer, n_neurons, n_batch_size}\n')

        val_loss = []
        for j, (train_index, val_index) in enumerate(kfold.split(train_X)):
            val_train_X = train_X.iloc[train_index, :]
            val_train_y = train_y.iloc[train_index]

            model = create_and_train_model(val_train_X, val_train_y, n_neurons, n_batch_size, 0.2, additional_layer)

            val_X = train_X.iloc[val_index, :].values.reshape((-1, 1, train_X.shape[1]))
            val_y = train_y.iloc[val_index].values

            val_accuracy = model.evaluate(val_X, val_y, verbose=0)
            val_loss.append(val_accuracy[1])

            print(f'{j+1}-FOLD ====> val acc: {val_accuracy[1]}')

        mean_val_loss = np.mean(val_loss)
        print(f'Mean validation loss: {mean_val_loss}')
        losses[i] = mean_val_loss

    best_index = np.argmin(losses)
    print(f"Best hyperparameters: {possible_combinations[best_index]} with validation RMSE: {losses[best_index]}")

    return possible_combinations[best_index], losses[best_index]

# Applying the functions

def main():
    train_scaled_df, test_scaled_df = preprocess_data()
    train_X, train_y, test_X, test_y = get_data_splits(train_scaled_df, test_scaled_df)
    model = create_and_train_model(train_X, train_y, test_X, test_y, n_neurons, n_batch_size, dropout, additional_layer)
    train_X2 = train_X.values.reshape((train_X.shape[0], 1, train_X.shape[1]))
    train_y2 = train_y.values.reshape((train_y.shape[0]))
    test_X2 = test_X.values.reshape((test_X.shape[0], 1, test_X.shape[1]))
    test_y2 = test_y.values
    #best_hyperparameters, best_loss = evaluate_hyperparameters(train_X, train_y)
    train_accuracy = model.evaluate(train_X2, train_y2, verbose=1)
    test_accuracy = model.evaluate(test_X2, test_y2, verbose=1)
    return train_accuracy, test_accuracy

# If this script is being run as the main module, execute the main function
if __name__ == "__main__":

    additional_layer = True
    n_neurons = 64
    n_batch_size = 32
    combination_no = 9
    dropout=0.2

    f_name = f'best_model{combination_no}.hdf5'

    train_accuracy, test_accuracy = main()
    print(f"train_accuracy: {train_accuracy}")
    print(f"test_accuracy: {test_accuracy}")

C:\Users\sedag\AppData\Local\Temp\ipykernel_28220\1501782592.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_train.sort_values(by='date', inplace=True)
C:\Users\sedag\AppData\Local\Temp\ipykernel_28220\1501782592.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data_test.sort_values(by='date', inplace=True)


------done get_data_splits---------------
Epoch 1/100
1868/1868 [==============================] - 29s 9ms/step - loss: 0.2291 - mae: 0.2291 - accuracy: 0.0000e+00 - val_loss: 0.2188 - val_mae: 0.2188 - val_accuracy: 5.3551e-04 - lr: 0.0010
Epoch 2/100
1868/1868 [==============================] - 16s 8ms/step - loss: 0.2285 - mae: 0.2285 - accuracy: 0.0000e+00 - val_loss: 0.2187 - val_mae: 0.2187 - val_accuracy: 5.3551e-04 - lr: 0.0010
Epoch 3/100
1868/1868 [==============================] - 16s 8ms/step - loss: 0.2283 - mae: 0.2283 - accuracy: 0.0000e+00 - val_loss: 0.2183 - val_mae: 0.2183 - val_accuracy: 5.3551e-04 - lr: 0.0010
Epoch 4/100
1868/1868 [==============================] - 16s 9ms/step - loss: 0.2282 - mae: 0.2282 - accuracy: 0.0000e+00 - val_loss: 0.2179 - val_mae: 0.2179 - val_accuracy: 5.3551e-04 - lr: 0.0010
Epoch 5/100
1868/1868 [==============================] - 15s 8ms/step - loss: 0.2280 - mae: 0.2280 - accuracy: 0.0000e+00 - val_loss: 0.2177 - val_mae: 0.2177 - v

1868/1868 [==============================] - 15s 8ms/step - loss: 0.2272 - mae: 0.2272 - accuracy: 0.0000e+00 - val_loss: 0.2194 - val_mae: 0.2194 - val_accuracy: 5.3551e-04 - lr: 6.4000e-08
Epoch 42/100
1868/1868 [==============================] - 15s 8ms/step - loss: 0.2272 - mae: 0.2272 - accuracy: 0.0000e+00 - val_loss: 0.2194 - val_mae: 0.2194 - val_accuracy: 5.3551e-04 - lr: 6.4000e-08
Epoch 43/100
1868/1868 [==============================] - 15s 8ms/step - loss: 0.2272 - mae: 0.2272 - accuracy: 0.0000e+00 - val_loss: 0.2194 - val_mae: 0.2194 - val_accuracy: 5.3551e-04 - lr: 6.4000e-08
Epoch 44/100
1868/1868 [==============================] - 15s 8ms/step - loss: 0.2272 - mae: 0.2272 - accuracy: 0.0000e+00 - val_loss: 0.2194 - val_mae: 0.2194 - val_accuracy: 5.3551e-04 - lr: 6.4000e-08
Epoch 45/100
1868/1868 [==============================] - 15s 8ms/step - loss: 0.2272 - mae: 0.2272 - accuracy: 0.0000e+00 - val_loss: 0.2194 - val_mae: 0.2194 - val_accuracy: 5.3551e-04 - lr: 6.40

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model

model = load_model('best_model9.hdf5')
all_data_train, all_data_test = preprocess_data()

X_train = all_data_train.drop(columns=['power/land', 'name', 'date'])
y_train = all_data_train['power/land']
X_test = all_data_test.drop(columns=['power/land', 'name', 'date'])
y_test = all_data_test['power/land']
test_X_values = X_test.values  # DataFrame을 NumPy 배열로 변환
test_X_reshaped = test_X_values.reshape((test_X_values.shape[0], 1, test_X_values.shape[1])) 


yhat = model.predict(test_X_reshaped)
X_test['pred'] = yhat

for location in test_data['name'].unique():
    print(f"Testing for location {location}...")

    location_data = X_test[X_test['name']==location]
    pred = location_data['pred']
    actual = location_data['power/land']

    mae = mean_absolute_error(actual, pred)
    print(f"mae is {mae}")

    location_data['date'] = pd.to_datetime(location_data['date'])
    location_data.set_index('date', inplace = True)

    daily_actual_sum = location_data.resample('D').sum()['power/land']
    daily_predicted_sum = location_data.resample('D').sum()['pred']

    corr_coefficient, p_value = calculate_pearsonr(daily_actual_sum, daily_predicted_sum)

    plt.figure(figsize=(15,6))
    plt.plot(daily_actual_sum, label = 'actual', color = 'red')
    plt.plot(daily_predicted_sum, label = 'predicted', color = 'blue')
    plt.title(f"in {location} pred/actual")
    plt.ylabel('power')
    plt.legend()

    corr_text = f"Pearson correlation coefficient: {corr_coefficient:.6f}\nP-value: {p_value:.6f}"
    plt.annotate(corr_text, xy=(0.75, 0.15), xycoords='axes fraction', fontsize=10, bbox=dict(boxstyle="round,pad=0.3", edgecolor="black", facecolor="aliceblue"))

    plt.show()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd5 in position 29: invalid continuation byte